In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


2.13.0-dev20230410


In [12]:
# get data files
train_set=pd.read_csv("train-data.tsv", sep='\t', header=None)
test_set=pd.read_csv("valid-data.tsv", sep='\t', header=None)

In [13]:
train_set.shape

(4179, 2)

In [14]:
train_set.columns

Int64Index([0, 1], dtype='int64')

In [15]:
test_set.columns

Int64Index([0, 1], dtype='int64')

In [16]:
train_set.head()

,0,1
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [46]:
train_set[0]=train_set[0].apply(lambda x: 1 if x=="ham" else 0)
test_set[0]=test_set[0].apply(lambda x: 1 if x=="ham" else 0)

In [47]:
train_set.head()

,0,1
0,1,ahhhh...just woken up!had a bad dream about u ...
1,1,you can never do nothing
2,1,"now u sound like manky scouse boy steve,like! ..."
3,1,mum say we wan to go then go... then she can s...
4,1,never y lei... i v lazy... got wat? dat day ü ...


In [49]:
y_train=train_set[0].values
x_train=train_set[1].values
x_test=test_set[1].values
y_test=test_set[0].values

In [50]:
x_train

array(['ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.',
       'you can never do nothing',
       'now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?',
       ...,
       'free entry into our £250 weekly competition just text the word win to 80086 now. 18 t&c www.txttowin.co.uk',
       '-pls stop bootydelious (32/f) is inviting you to be her friend. reply yes-434 or no-434 see her: www.sms.ac/u/bootydelious stop? send stop frnd to 62468',
       "tell my  bad character which u dnt lik in me. i'll try to change in  &lt;#&gt; . i ll add tat 2 my new year resolution. waiting for ur reply.be frank...good morning."],
      dtype=object)

In [51]:
from keras.preprocessing.text import Tokenizer

In [52]:
tokenizer=Tokenizer(oov_token="nothing")

In [53]:
tokenizer.fit_on_texts(x_train)

In [54]:
tokenizer.word_index

{'nothing': 341,
 'i': 2,
 'to': 3,
 'you': 4,
 'a': 5,
 'the': 6,
 'u': 7,
 'and': 8,
 'in': 9,
 'is': 10,
 'me': 11,
 'my': 12,
 'for': 13,
 'your': 14,
 'it': 15,
 'of': 16,
 'call': 17,
 'have': 18,
 'on': 19,
 'now': 20,
 'that': 21,
 '2': 22,
 'are': 23,
 'so': 24,
 'but': 25,
 'not': 26,
 'or': 27,
 'do': 28,
 'at': 29,
 'can': 30,
 "i'm": 31,
 'will': 32,
 'be': 33,
 'with': 34,
 'if': 35,
 'get': 36,
 'just': 37,
 'ur': 38,
 'no': 39,
 'this': 40,
 'we': 41,
 '4': 42,
 'gt': 43,
 'lt': 44,
 'up': 45,
 'ok': 46,
 'go': 47,
 'free': 48,
 'when': 49,
 'how': 50,
 'out': 51,
 'all': 52,
 'from': 53,
 'what': 54,
 'know': 55,
 'like': 56,
 'then': 57,
 'good': 58,
 'got': 59,
 'come': 60,
 'its': 61,
 'time': 62,
 'day': 63,
 'only': 64,
 'am': 65,
 'was': 66,
 'he': 67,
 'love': 68,
 'send': 69,
 'there': 70,
 'as': 71,
 'text': 72,
 'txt': 73,
 'want': 74,
 'going': 75,
 'one': 76,
 'by': 77,
 'ü': 78,
 'about': 79,
 'need': 80,
 'still': 81,
 'lor': 82,
 'n': 83,
 'r': 84,
 "i'l

In [55]:
sequences=tokenizer.texts_to_sequences(x_train)

In [56]:
from keras.utils import pad_sequences

In [57]:
sequences=pad_sequences(sequences, padding="post", maxlen=50)

In [58]:
sequences.shape

(4179, 50)

In [59]:
l=len(tokenizer.word_index)
l

7692

In [70]:
tf.config.run_functions_eagerly(True)

In [83]:
import tensorflow_hub as hub
use = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',trainable=False,dtype=tf.string,input_shape=[])

model=tf.keras.Sequential([
    tf.keras.layers.Embedding(l,output_dim=2, input_length=50),
    tf.keras.layers.SimpleRNN(32, activation='tanh',return_sequences=False),
    keras.Input(shape=[], dtype = tf.string),
      use,
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

KeyboardInterrupt: 

In [79]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [80]:
model.fit(sequences, y_train, epochs=1)

Epoch 1/15
127/131 [============================>.] - ETA: 0s - loss: 0.4343 - accuracy: 0.8533

InvalidArgumentError: Exception encountered when calling layer 'embedding_5' (type Embedding).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[26,25] = 7692 is not in [0, 7692) [Op:ResourceGather] name: 

Call arguments received by layer 'embedding_5' (type Embedding):
  • inputs=tf.Tensor(shape=(32, 50), dtype=float32)